# [Projet 6: Catégorisez automatiquement des questions](https://openclassrooms.com/fr/projects/categorisez-automatiquement-des-questions)
(parcours data: [here](https://openclassrooms.com/paths/63-data-scientist))

## Intelligence

Resources:
- [tutorial on kaggle](https://www.kaggle.com/c/word2vec-nlp-tutorial#part-1-for-beginners-bag-of-words) (I did this basically on my own, but look at the features with scikit-learn section)

### Imports

In [1]:
import os
HOME = os.path.expanduser('~/')
HOST = os.uname()[1]
if HOST == 'Arthurs-MacBook-Pro.local':
    os.chdir(HOME+'Documents/GitHub/OCDataSciencePath/Project6/Work/')    # @home
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [3]:
from Pkg.helper import avg_jacard, hamming_score # only works if correct working dirrectory

### Data

In [4]:
if HOST == 'Arthurs-MacBook-Pro.local':
    pathToDataDir = HOME+'Documents/Dropbox/Transit/OCDataScienceData/Project6/Data/'    # @home
    pathToIntellDir = HOME+'Documents/Dropbox/Transit/OCDataScienceData/Project6/Intelligence/'    # @home
else:
    raise ValueError('unknown host: {}'.format(HOST))

In [5]:
filename = '100k_forLearning.npz'

tmp = np.load(pathToDataDir+filename)
x_train,x_test,y_train,y_test = tmp['arr_0'],tmp['arr_1'],tmp['arr_2'],tmp['arr_3']

In [6]:
x_train,x_train.shape

(array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]), (89816, 597))

In [7]:
x_test,x_test.shape

(array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]), (22444, 597))

In [8]:
y_train,y_train.shape

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), (89816, 37))

In [9]:
y_test,y_test.shape

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), (22444, 37))

### Try things

Use [OneVsRest](http://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) to perform multi-label (as in [here](https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5),...)

**Notes:**
- the tags are heavily imbalanced -> predicts 0 if we do nothing for it

In [10]:
x_t,x_v,y_t,y_v = train_test_split(x_train,y_train,test_size=0.2,random_state=8)

In [11]:
ovr = OneVsRestClassifier(LinearSVC(penalty='l2',
                                    loss='hinge',
                                    C=1, # 0.1, #1.0,
                                    fit_intercept=True,
                                    class_weight='balanced',
                                    random_state=8))

ovr.fit(x_t,y_t)

OneVsRestClassifier(estimator=LinearSVC(C=1, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=8, tol=0.0001, verbose=0),
          n_jobs=1)

In [17]:
i = 50000
x_v_ = x_v[:i,:]
y_v_ = y_v[:i,:]

y_hat = ovr.predict(x_v_)

In [18]:
y_hat

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [19]:
y_v_

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

*Some [metrics](https://en.wikipedia.org/wiki/Multi-label_classification#Statistics_and_evaluation_metrics):*

In [20]:
print(avg_jacard(y_v_,y_hat))

0.352921089586


In [21]:
print(hamming_score(y_v_,y_hat))

0.352921089586


In [37]:
# save model for later use
model = ovr
pickle.dump(model, open(pathToIntellDir+'model.pkl','wb'))

# DRAFT -------

## Decrypt

In [ ]:
vocab_tag = pd.read_csv(pathToDataDir+'vocab_tag.csv',index_col=[0])
vocab_tag

In [ ]:
k = 8

tag = []
for i,b in enumerate(y_[k,:]):
    if b:
        tag.append(vocab_tag.loc[i,'token'])
tag

In [ ]:
# for category in categories:
#     print('... Processing {}'.format(category))
#     # train the model using X_dtm & y
#     ovr.fit(X_train, train[category])
#     # compute the testing accuracy
#     prediction = SVC_pipeline.predict(X_test)
#     print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))